In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data_Q3_2019/2019-07-01.csv')

In [2]:
df.head()

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2019-07-01,Z305B2QN,ST4000DM000,4000787030016,0,119.0,224077128.0,NaN,NaN,91.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-07-01,ZJV0XJQ4,ST12000NM0007,12000138625024,0,83.0,196781888.0,NaN,NaN,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-07-01,ZJV0XJQ3,ST12000NM0007,12000138625024,0,76.0,37537312.0,NaN,NaN,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-07-01,ZJV0XJQ0,ST12000NM0007,12000138625024,0,100.0,1592144.0,NaN,NaN,93.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-07-01,PL1331LAHG1S4H,HGST HMS5C4040ALE640,4000787030016,0,100.0,0.0,134.0,103.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.shape

(110638, 129)

In [4]:
df.model.unique()

array(['ST4000DM000', 'ST12000NM0007', 'HGST HMS5C4040ALE640',
       'ST8000NM0055', 'ST8000DM002', 'HGST HMS5C4040BLE640',
       'TOSHIBA MG07ACA14TA', 'HGST HUH721212ALN604',
       'HGST HUH721212ALE600', 'TOSHIBA MQ01ABF050', 'ST500LM030',
       'ST6000DX000', 'ST10000NM0086', 'TOSHIBA MQ01ABF050M',
       'WDC WD5000LPVX', 'ST500LM012 HN', 'HGST HUH728080ALE600',
       'TOSHIBA MD04ABA400V', 'TOSHIBA HDWF180', 'ST8000DM005',
       'Seagate SSD', 'ST4000DM005', 'WDC WD5000LPCX',
       'HGST HDS5C4040ALE630', 'DELLBOSS VD', 'ST500LM021',
       'Hitachi HDS5C4040ALE630', 'HGST HUS726040ALE610',
       'Seagate BarraCuda SSD ZA500CM10002',
       'Seagate BarraCuda SSD ZA2000CM10002', 'HGST HUH721010ALE600',
       'TOSHIBA HDWE160', 'WDC WD5000BPKT', 'ST6000DM001', 'WDC WD60EFRX',
       'ST320LT007', 'ST8000DM004', 'ST9250315AS', 'HGST HMS5C4040BLE641',
       'ST1000LM024 HN', 'ST6000DM004'], dtype=object)

In [5]:
df.capacity_bytes.sort_values().unique()

array([  137438952960,   250059350016,   320072933376,   480036847616,
         500107862016,  1000204886016,  2000398934016,  4000787030016,
        6001175126016,  8001563222016, 10000831348736, 12000138625024,
       14000519643136])

In [6]:
df.capacity_bytes.describe()

count    1.106380e+05
mean     8.286890e+12
std      3.602449e+12
min      1.374390e+11
25%      4.000787e+12
50%      8.001563e+12
75%      1.200014e+13
max      1.400052e+13
Name: capacity_bytes, dtype: float64

In [14]:
def get_manufacturer(model):
    if model.startswith('ST'):
        return 'seagate'
    elif model.startswith('Sea'):
        return 'seagate'
    elif model.startswith('HGST'):
        return 'hitachi'
    elif model.startswith('Hit'):
        return 'hitachi'
    elif model.startswith('TOS'):
        return 'toshiba'
    elif model.startswith('DEL'):
        return 'dell'
    elif model.startswith('WD'):
        return 'western digital'
    else:
        return 'unknown'

In [15]:
df['manufacturer'] = df.model.apply(get_manufacturer)
df[['model', 'manufacturer']].head()

,model,manufacturer
0,ST4000DM000,seagate
1,ST12000NM0007,seagate
2,ST12000NM0007,seagate
3,ST12000NM0007,seagate
4,HGST HMS5C4040ALE640,hitachi


In [16]:
df.manufacturer.isnull().sum()

0

In [17]:
df.manufacturer.value_counts()

seagate            81521
hitachi            26597
toshiba             2220
western digital      280
dell                  20
Name: manufacturer, dtype: int64

In [19]:
# list comp. to find index of a value in a series/column
# df['manufacturer'].loc[lambda x: x=='unknown'].index

In [20]:
#df.iloc[5913]

- The following cells are for my reference they are misc. commands

In [ ]:
# to view all columns instead of ......
# cols = df.columns.tolist()
# cols

# to view stats on SMART STAT
# df.smart_9_raw.describe()

# to view nulls because overall count is different
# df.smart_9_raw.isnull().sum()

# to view all non-null columns in a record
# df.iloc[54718].dropna()

# to view 2 columns for a specific model and its SMART STAT value
# df[['model','smart_9_raw']][df.model=='ST12000NM0007']

# David's function
#def get_manufacturer(df):
    models = df['model'].str[0:-1]
    manufacturers = []
    for i in models:
        if i[0:2] == 'ST':
            manufacturers.append('Seagate')
        else:
            manufacturers.append('Unknown')
    df['manufacturer'] = manufacturers
    return df  